 Object Detection and Tracking Project

In [2]:
# Install dependencies
!pip install ultralytics opencv-python filterpy

import cv2
import torch
from ultralytics import YOLO
import numpy as np
from filterpy.kalman import KalmanFilter

# -------------------------------
# 1. Load YOLOv8 model (pre-trained on COCO dataset)
# -------------------------------
model = YOLO("yolov8n.pt")  # small version for speed

# -------------------------------
# 2. SORT Tracker Implementation
# -------------------------------
class SortTracker:
    def __init__(self):
        self.trackers = []
        self.id_count = 0

    def update(self, detections):
        updated_tracks = []
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            # In a real SORT implementation, you would use Kalman filters
            # to predict and associate detections with existing tracks.
            # For this example, we'll just return the detections as tracks.
            updated_tracks.append((int(x1), int(y1), int(x2), int(y2), int(cls)))
        return updated_tracks

tracker = SortTracker()

# -------------------------------
# 3. Open Video Source
# -------------------------------
# 0 = webcam, or provide video file path
# cap = cv2.VideoCapture(0)  # change to "video.mp4" if you want file
cap = cv2.VideoCapture("/content/video.mp4") # Assuming you have a video file named video.mp4 in /content

# Get video properties for saving the output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can use other codecs like 'XVID'
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # -------------------------------
    # 4. Run YOLO detection on frame
    # -------------------------------
    results = model(frame, verbose=False)[0]

    detections = []
    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        detections.append([x1, y1, x2, y2, conf, int(cls)])

    # -------------------------------
    # 5. Update tracker
    # -------------------------------
    tracks = tracker.update(detections)

    # -------------------------------
    # 6. Draw results and write to output video
    # -------------------------------
    for track in tracks:
        x1, y1, x2, y2, cls = track
        label = model.names[cls]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    # Write the frame to the output video
    out.write(frame)

    # # press q to quit - not needed when saving to file
    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break

cap.release()
out.release()
# cv2.destroyAllWindows() # not needed when saving to file

In [4]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Open Video Source
# Assuming you have a video file named video.mp4 in /content
cap = cv2.VideoCapture("/content/video.mp4")

# Get video properties for saving the output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can use other codecs like 'XVID'
out = cv2.VideoWriter('output_video_tk_fix.mp4', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection on frame
    results = model(frame, verbose=False)[0]

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        label = model.names[int(cls)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        cv2.putText(frame, label, (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()

print("Processing complete. Output video saved as output_video_tk_fix.mp4")

Processing complete. Output video saved as output_video_tk_fix.mp4


In [5]:
!pip install ultralytics gradio opencv-python-headless

import cv2
import gradio as gr
from ultralytics import YOLO

# YOLOv8
model = YOLO("yolov8n.pt")

def detect_objects(image):
    results = model(image, verbose=False)[0]
    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        label = f"{model.names[int(cls)]} {conf:.2f}"
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        cv2.putText(image, label, (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    return image

#  Gradio
demo = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="numpy", label="Upload Image"),
    outputs=gr.Image(type="numpy", label="Detected Objects"),
    title="YOLOv8 Object Detection",
    description="Upload an image and the model will detect objects in it."
)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7a34fc7fc3b363f2ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7a34fc7fc3b363f2ad.gradio.live
